In [1]:
# import necessary packages and libraries
import json, datetime, time

import pandas as pd

from azure.storage.blob import BlobServiceClient, BlobClient

from pyspark.sql.types import *

from pyspark.sql.functions import *

import com.microsoft.spark.sqlanalytics
from com.microsoft.spark.sqlanalytics.Constants import Constants

In [2]:
# define necessary connections to storage ( source and destination )
# abfss because my storage account has HNS enabled
adlsAcct = "sparkmtndatalake.dfs.core.usgovcloudapi.net/"
# SAS Token Expiration : 27-April-2023
adlsSas = "?sv=2021-12-02&ss=b&srt=sco&sp=rwdlacx&se=2023-04-27T22:50:09Z&st=2023-03-28T14:50:09Z&spr=https&sig=f74oVHLZZ0IyNULznbqdgLeF50aCHn8rnegQdG6VcOs%3D"
adlsCont = "insiders"

# define variables for blob store connection
storage_acct = "sparkmtndatalake"
container_name = "insiders"
linked_svc = "SparkMtnLake"

# sas token will be pulled from linked service definition
sas_token = mssparkutils.credentials.getConnectionStringOrCreds( linked_svc )

spark.conf.set( "fs.azure.sas.%s.%s.dfs.core.usgovcloudapi.net" % ( container_name, storage_acct), sas_token )

httpsUrl = "https://%s.blob.core.usgovcloudapi.net/" % ( storage_acct )
abfssUrl = "abfss://%s@%s.dfs.core.usgovcloudapi.net/" % ( container_name, storage_acct )

rawFldr = "raw/"

blobSvcConn = BlobServiceClient( httpsUrl, credential = sas_token )

contClient = blobSvcConn.get_container_client( container_name )

firstFilePath = rawFldr + "007a4ada-4bfe-4e15-a180-b5844dd2d275/"

In [ ]:
# files to load for dim tables

filesToLoad = []

for item in contClient.walk_blobs( rawFldr ) :

    # print( item.name )

    for subItem in contClient.walk_blobs( item.name ) :

        # print( subItem.name )

        subItemSplit = ( subItem.name ).split( "/" )

        fileName = subItemSplit[ -1 ]

        if fileName.upper() not in filesToLoad and fileName.upper() not in [ "CLAIM.NDJSON", "OBSERVATION.NDJSON", "PATIENT.NDJSON" ] :

            filesToLoad.append( ( subItem.name ).upper() )

print( filesToLoad )

In [ ]:
# test two files with naming convention : 
# hospitalInformation
# practitionerInformation

fileToDfName = {}

firstFileList = []

for item in contClient.walk_blobs( firstFilePath ) :

    # print( item.name )

    itemFullName = item.name

    schemaPath = abfssUrl + itemFullName

    itemSplit = itemFullName.split( "/" )

    fileName = itemSplit[ -1 ]

    fileSplit = fileName.split( "." )

    if fileName not in fileToDfName.keys() :

        if "hospitalInformation" in fileName :

            dfName = "df_hospitalInformation"
            schemaName = "sch_hospitalInformation"

        elif "practitionerInformation" in fileName :

            dfName = "df_practitionerInformation"
            schemaName = "sch_practitionerInformation"

        else :

            dfName = "df_" + fileSplit[ 0 ]
            schemaName = "sch_" + fileSplit[ 0 ]

        itemsList = []
        itemsList.append( dfName )
        itemsList.append( schemaName )

        fileToDfName[ fileName ] = itemsList

print( fileToDfName )

In [15]:
# spark.read.option( "multiline", "true" ).json( pathClaimSchema ).schema
# spark.read.option( "multiline", "true" ).option( "columnNameOfCorruptRecord", "corruptRecord" ).schema( claimSchema ).json( filesToLoadList )

for file in fileToDfName.keys() :

    schemaPath = abfssUrl + firstFilePath + file

    dfName = fileToDfName[ file ][ 0 ]
    schName = fileToDfName[ file ][ 1 ]

    dynCmdStr = schName + " = spark.read.option( 'multiline', 'true' ).json( '" + schemaPath + "' ).schema"

    print( dynCmdStr )
    exec( dynCmdStr )

    if 'hospitalInformation' in file :

        hiFileList = []

        for fileToLoad in filesToLoad :

            if 'hospitalInformation' in fileToLoad :

                fileString = abfssUrl + fileToLoad

                hiFileList.append( fileString )

        dynCmdStr = dfName + " = spark.read.option( 'multiline', 'true' ).option( 'columnNameOfCorruptRecord', 'corruptRecord' ).schema( " + schName + " ).json( hiFileList )"

        print( dynCmdStr )
        exec( dynCmdStr )

    elif 'practitionerInformation' in file :

        piFileList = []

        for fileToLoad in filesToLoad :

            if 'practitionerInformation' in fileToLoad :

                fileString = abfssUrl + fileToLoad

                piFileList.append( fileString )

        dynCmdStr = dfName + " = spark.read.option( 'multiline', 'true' ).option( 'columnNameOfCorruptRecord', 'corruptRecord' ).schema( " + schName + " ).json( piFileList )"

        print( dynCmdStr )
        exec( dynCmdStr )

    else :

        filePath = abfssUrl + rawFldr + "*/" + file

        dynCmdStr = dfName + " = spark.read.option( 'multiline', 'true' ).option( 'columnNameOfCorruptRecord', 'corruptRecord' ).schema( " + schName + " ).json( '" + filePath + "' )"

        print( dynCmdStr )
        exec( dynCmdStr )

    # dynamic cmd to write dataframe contents to dedicated pool table in 'dim' schema

    dfSplit = dfName.split( "_" )

    tblName = dfSplit[ -1 ]

    dynCmdStr = "( " + dfName + ".write.option( Constants.SERVER, '<dedicated pool instance url>' ).option( Constants.USER, 'sqladminuser' ).option( Constants.PASSWORD, '<sqladminuser_pwd>' ).option( Constants.STAGING_STORAGE_ACCOUNT_KEY, '<storage account key>' ).option( Constants.TEMP_FOLDER, '<abfss url for storage account, container and parquet file write folder>' ).mode( 'overwrite' ).synapsesql( '<sqldb>.dim." + tblName + "' ) )"
    print( dynCmdStr )
    # exec( dynCmdStr )